In [1]:
import matplotlib.pyplot as plt
import time
import os
import pandas as pd
import datetime
import seaborn as sns

In [2]:
pricepath = "C:/Users/TR/shell datathon/price.csv"
testpath = "C:/Users/TR/shell datathon/test.csv"
trainpath = "C:/Users/TR/shell datathon/train.csv"
if(os.path.exists(pricepath)):
    priceData = pd.read_csv(pricepath)
if(os.path.exists(testpath)):
    testData = pd.read_csv(testpath)
if(os.path.exists(trainpath)):
    trainData = pd.read_csv(trainpath)

In [3]:
trainData = trainData.loc[:, trainData.columns!='Plant']

In [4]:
trainData.head()
trainData['Billing Date'] = trainData['Billing Date'].astype('string')
trainData

,Customer,Area,City Code,Billing Date,Product,Litres
0,10048261,Karadeniz Bölgesi,883,2014-01-01,Benzin,0.0
1,10048261,Karadeniz Bölgesi,883,2014-01-01,Benzin,0.0
2,10048261,Karadeniz Bölgesi,883,2014-01-01,Motorin,0.0
3,10048261,Karadeniz Bölgesi,883,2014-01-01,Motorin,0.0
4,10048261,Karadeniz Bölgesi,883,2014-01-02,Motorin,16685.0
...,...,...,...,...,...,...
1379255,12663344,Doğu Anadolu Bölgesi,218,2018-12-22,Motorin,1504.0
1379256,12663344,Doğu Anadolu Bölgesi,218,2018-12-28,Benzin,11111.0
1379257,12663344,Doğu Anadolu Bölgesi,218,2018-12-28,Motorin,22882.0
1379258,12664221,Marmara Bölgesi,941,2018-12-29,Benzin,6528.0


In [5]:
priceDataTest = priceData[priceData["Validfrom"]>"2019"]
priceDataTrain = priceData[priceData["Validfrom"]<"2019"]

mergedData = pd.merge(trainData, priceDataTrain, how="left", left_on = ["City Code","Billing Date","Product"], right_on = ["City Code","Validfrom","Product"]).drop(["Validfrom"],axis=1)
mergedData.head()

,Customer,Area,City Code,Billing Date,Product,Litres,CRP
0,10048261,Karadeniz Bölgesi,883,2014-01-01,Benzin,0.0,4.98
1,10048261,Karadeniz Bölgesi,883,2014-01-01,Benzin,0.0,4.98
2,10048261,Karadeniz Bölgesi,883,2014-01-01,Motorin,0.0,4.58
3,10048261,Karadeniz Bölgesi,883,2014-01-01,Motorin,0.0,4.58
4,10048261,Karadeniz Bölgesi,883,2014-01-02,Motorin,16685.0,4.65


In [47]:
mergedData['Billing Date'] = mergedData['Billing Date'].astype('datetime64[ns]')
BCPL = mergedData.drop("CRP", axis=1)
BCPC = mergedData.drop("Litres", axis=1)
BCPC

,Customer,Area,City Code,Billing Date,Product,CRP
0,10048261,Karadeniz Bölgesi,883,2014-01-01,Benzin,4.98
1,10048261,Karadeniz Bölgesi,883,2014-01-01,Benzin,4.98
2,10048261,Karadeniz Bölgesi,883,2014-01-01,Motorin,4.58
3,10048261,Karadeniz Bölgesi,883,2014-01-01,Motorin,4.58
4,10048261,Karadeniz Bölgesi,883,2014-01-02,Motorin,4.65
...,...,...,...,...,...,...
1379255,12663344,Doğu Anadolu Bölgesi,218,2018-12-22,Motorin,6.11
1379256,12663344,Doğu Anadolu Bölgesi,218,2018-12-28,Benzin,6.01
1379257,12663344,Doğu Anadolu Bölgesi,218,2018-12-28,Motorin,5.83
1379258,12664221,Marmara Bölgesi,941,2018-12-29,Benzin,5.92


In [50]:
CRP_data = BCPC.groupby(["Customer","Product", "Area", "City Code"]).resample('W-Wed', label='right', closed = 'right', on='Billing Date').mean()
CRP_data = CRP_data.drop(["Customer","City Code"],axis=1).reset_index()
litres_data = BCPL.groupby(["Customer","Product", "Area", "City Code"]).resample('W-Wed', label='right', closed = 'right', on='Billing Date').sum()
litres_data = litres_data.drop(["Customer","City Code"],axis=1).reset_index()
CRP_data

,Customer,Product,Area,City Code,Billing Date,CRP
0,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-01,4.980
1,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-08,5.070
2,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-15,4.980
3,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-22,4.980
4,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-29,5.045
...,...,...,...,...,...,...
530883,12663344,Motorin,Doğu Anadolu Bölgesi,218,2018-12-19,6.110
530884,12663344,Motorin,Doğu Anadolu Bölgesi,218,2018-12-26,6.110
530885,12663344,Motorin,Doğu Anadolu Bölgesi,218,2019-01-02,5.830
530886,12664221,Benzin,Marmara Bölgesi,941,2019-01-02,5.920


In [51]:
weekly_data = pd.concat([CRP_data, litres_data["Litres"]], axis=1, join="inner")
weekly_data.sort_values(by=["Customer", "Billing Date", "Product"])
weekly_data['Billing Date'] = weekly_data['Billing Date'].astype('string')
weekly_data

,Customer,Product,Area,City Code,Billing Date,CRP,Litres
0,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-01,4.980,0.0
1,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-08,5.070,9282.0
2,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-15,4.980,4179.0
3,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-22,4.980,15234.0
4,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-29,5.045,9206.0
...,...,...,...,...,...,...,...
530883,12663344,Motorin,Doğu Anadolu Bölgesi,218,2018-12-19,6.110,55062.0
530884,12663344,Motorin,Doğu Anadolu Bölgesi,218,2018-12-26,6.110,1504.0
530885,12663344,Motorin,Doğu Anadolu Bölgesi,218,2019-01-02,5.830,22882.0
530886,12664221,Benzin,Marmara Bölgesi,941,2019-01-02,5.920,6528.0


In [64]:
y = weekly_data.Litres
weekly_data.drop(['Litres'], axis=1, inplace=True)
X = weekly_data
X

,Customer,Product,Area,City Code,Billing Date,CRP
0,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-01,4.980
1,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-08,5.070
2,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-15,4.980
3,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-22,4.980
4,10048261,Benzin,Karadeniz Bölgesi,883,2014-01-29,5.045
...,...,...,...,...,...,...
530883,12663344,Motorin,Doğu Anadolu Bölgesi,218,2018-12-19,6.110
530884,12663344,Motorin,Doğu Anadolu Bölgesi,218,2018-12-26,6.110
530885,12663344,Motorin,Doğu Anadolu Bölgesi,218,2019-01-02,5.830
530886,12664221,Benzin,Marmara Bölgesi,941,2019-01-02,5.920


In [65]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)
X_train.head()

,Customer,Product,Area,City Code,Billing Date,CRP
452587,12227003,Benzin,Akdeniz Bölgesi,990,2015-10-21,4.400000
154681,10049404,Motorin,Marmara Bölgesi,835,2016-02-17,3.372000
250363,10049731,Motorin,Marmara Bölgesi,835,2018-10-03,6.311429
354069,10054976,Motorin,Marmara Bölgesi,835,2016-12-07,4.330000
24244,10048935,Motorin,Marmara Bölgesi,288,2017-04-19,4.640000


In [66]:
object_cols=["Area", "Product"]
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

In [67]:
X_train = pd.get_dummies(X_train, columns=low_cardinality_cols)
X_valid = pd.get_dummies(X_valid, columns=low_cardinality_cols)

In [68]:
str_to_time = lambda str_time: time.mktime(datetime.datetime.strptime(str_time,"%Y-%m-%d").timetuple())
X_train["Billing Date"] = X_train["Billing Date"].apply(str_to_time)
str_to_time = lambda str_time: time.mktime(datetime.datetime.strptime(str_time,"%Y-%m-%d").timetuple())
X_valid["Billing Date"] = X_valid["Billing Date"].apply(str_to_time)

In [69]:
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns
final_X_train

,Customer,City Code,Billing Date,CRP,Area_Akdeniz Bölgesi,Area_Doğu Anadolu Bölgesi,Area_Ege Bölgesi,Area_Güneydoğu Anadolu Bölgesi,Area_Karadeniz Bölgesi,Area_Marmara Bölgesi,Area_İç Anadolu Bölgesi,Product_Benzin,Product_Motorin
0,12227003.0,990.0,1.445375e+09,4.400000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10049404.0,835.0,1.455656e+09,3.372000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,10049731.0,835.0,1.538514e+09,6.311429,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,10054976.0,835.0,1.481058e+09,4.330000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,10048935.0,288.0,1.492549e+09,4.640000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424705,10054992.0,902.0,1.486501e+09,5.420000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
424706,10049395.0,774.0,1.469567e+09,3.820000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
424707,10049266.0,367.0,1.415135e+09,4.396000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
424708,12146288.0,367.0,1.441141e+09,3.860000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [70]:
final_X_valid

,Customer,City Code,Billing Date,CRP,Area_Akdeniz Bölgesi,Area_Doğu Anadolu Bölgesi,Area_Ege Bölgesi,Area_Güneydoğu Anadolu Bölgesi,Area_Karadeniz Bölgesi,Area_Marmara Bölgesi,Area_İç Anadolu Bölgesi,Product_Benzin,Product_Motorin
0,12516293.0,611.0,1.474405e+09,4.780000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,10052367.0,884.0,1.442351e+09,4.650000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10049699.0,434.0,1.429650e+09,3.906667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,10049239.0,574.0,1.409692e+09,4.488333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,10049732.0,835.0,1.422392e+09,4.110000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106173,10048265.0,589.0,1.518556e+09,4.650000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
106174,12131791.0,367.0,1.415135e+09,4.270000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
106175,12175078.0,589.0,1.506460e+09,5.320000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
106176,12142311.0,915.0,1.479244e+09,4.790000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [71]:
y_valid

522878      5939.0
349972         0.0
241291     28716.0
109851    105488.0
250432     13596.0
            ...   
2039           0.0
428640     49091.0
440286      2963.0
432143      7008.0
482268         0.0
Name: Litres, Length: 106178, dtype: float64

In [77]:
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=1000, n_jobs = 6,early_stopping_rounds=5)
my_model.fit(final_X_train,y_train, eval_set=[(final_X_valid, y_valid)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=5, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=6,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [78]:
from sklearn.metrics import mean_absolute_error
#500 4357.81
#300 4463.24
#500 9754.000410298599
#600 4338.56
#700 4321.66
#750 4314.72
#800 9685.754502557666
#1000 9686.069229202925
#1500 4310.16
#2000 4310.16
predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 9686.069229202925


In [79]:
testData

,index,Customer,Area,City Code,Product,Week Start Date
0,0,10048261,Karadeniz Bölgesi,883,Motorin,2019-01-07
1,1,10048261,Karadeniz Bölgesi,883,Motorin,2019-01-14
2,2,10048261,Karadeniz Bölgesi,883,Motorin,2019-01-21
3,3,10048261,Karadeniz Bölgesi,883,Motorin,2019-01-28
4,4,10048261,Karadeniz Bölgesi,883,Motorin,2019-02-04
...,...,...,...,...,...,...
108467,108467,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-02
108468,108468,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-09
108469,108469,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-16
108470,108470,12739737,Doğu Anadolu Bölgesi,194,Benzin,2019-12-23
